# Fake News Classfier Using Bidirectional LSTM

## Importing Libraries

In [14]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

## Setting GPU for training model

In [15]:
physical_devices = tf.config.list_physical_devices('GPU')
try: 
    tf.config.experimental.set_memory_growth(physical_devices[0],True)
except:
    pass

In [16]:
print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Importing Dataset

In [17]:
dataset=pd.read_csv('train.csv/train.csv')

In [18]:
dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


## Data Preprocessing

In [19]:
dataset=dataset.dropna()

In [20]:
X=dataset.drop('label',axis=1)

In [21]:
y=dataset['label']

In [22]:
vocabulary_size=5000

In [23]:
messages=X.copy()

In [24]:
messages.reset_index(inplace=True)

In [25]:
import nltk
import re
from nltk.corpus import stopwords

In [26]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [28]:
onehot_repr=[one_hot(words,vocabulary_size)for words in corpus]

In [29]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

## Train - Test Split

In [30]:
import numpy as np

In [31]:
X=np.array(embedded_docs)
y=np.array(y)

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Training Bidirectional LSTM Model

In [56]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_vector_features, input_length=sent_length))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Bidirectional(LSTM(50)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [57]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_3 (Bidirectio  (None, 20, 200)          112800    
 nal)                                                            
                                                                 
 bidirectional_4 (Bidirectio  (None, 100)              100400    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                6464      
                                                                 
 dense_2 (Dense)             (None, 1)                

In [58]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 24ms/step - loss: 0.2790 - accuracy: 0.8689 - val_loss: 0.1960 - val_accuracy: 0.9180
Epoch 2/10
192/192 [==============================] - 3s 18ms/step - loss: 0.1411 - accuracy: 0.9467 - val_loss: 0.2057 - val_accuracy: 0.9175
Epoch 3/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0994 - accuracy: 0.9643 - val_loss: 0.2241 - val_accuracy: 0.9067
Epoch 4/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0749 - accuracy: 0.9749 - val_loss: 0.2582 - val_accuracy: 0.9097
Epoch 5/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0641 - accuracy: 0.9781 - val_loss: 0.3148 - val_accuracy: 0.9084
Epoch 6/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0470 - accuracy: 0.9844 - val_loss: 0.4077 - val_accuracy: 0.9054
Epoch 7/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0344 - accuracy: 0.9891 - val_loss: 0.3491 - val_accuracy: 0.9062

## Performance Metrics and Analytics

In [59]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

189/189 [==============================] - 2s 6ms/step


In [60]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [61]:
confusion_matrix(y_test,y_pred)

array([[3091,  328],
       [ 263, 2353]], dtype=int64)

In [62]:
accuracy_score(y_test,y_pred)

0.9020712510356255

In [63]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91      3419
           1       0.88      0.90      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035



In [64]:
import pickle

In [65]:
with open('classifier-model.pkl', 'wb') as file:
    pickle.dump(model, file)

INFO:tensorflow:Assets written to: ram://52515d56-7b93-4a50-955a-72d94068e8f1/assets


INFO:tensorflow:Assets written to: ram://52515d56-7b93-4a50-955a-72d94068e8f1/assets
